# VacationPy

In [18]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [19]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,philadelphia,39.9523,-75.1638,9.94,55,75,6.69,US,1699757038
1,1,revda,56.8010,59.9303,1.48,98,100,1.80,RU,1699757178
2,2,suileng,47.2500,127.0833,-7.90,79,14,2.76,CN,1699757178
3,3,waitangi,-43.9535,-176.5597,14.34,77,0,0.89,NZ,1699757179
4,4,ilulissat,69.2167,-51.1000,-6.99,49,75,8.23,GL,1699757180


In [20]:
# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng", "Lat",
                                      geo=True,
                                      tiles="OSM", 
                                      size= "Humidity", 
                                      color= "City",
                                      frame_width= 650,
                                      frame_height = 550)


# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [21]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df[(city_data_df["Max Temp"] >= 20) & 
                                  (city_data_df["Max Temp"] <= 23.89) & 
                                  (city_data_df["Wind Speed"] <= 4.5) &
                                  (city_data_df["Cloudiness"] <= 65) &
                                  (city_data_df["Humidity"] <= 60)]  
                                  

# Drop any rows with null values
ideal_city_data_df_clean = ideal_city_data_df.dropna(how='all')

# Display sample data
ideal_city_data_df_clean

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,pathalgaon,22.5667,83.4667,23.46,50,0,1.06,IN,1699757184
137,137,khairagarh,26.9500,77.8167,20.47,55,0,2.35,IN,1699757272
142,142,kidal,18.4411,1.4078,23.89,11,0,4.23,ML,1699757275
160,160,zouerate,22.7187,-12.4521,22.26,24,19,3.43,MR,1699757290
173,173,singtam,27.2333,88.5000,23.16,45,1,0.38,IN,1699757301
218,218,bairnsdale,-37.8333,147.6167,21.14,51,0,1.79,AU,1699757331
262,262,agadez,19.7500,10.2500,21.38,18,1,3.66,NE,1699757360
271,271,dourbali,11.8091,15.8632,23.81,54,35,2.31,TD,1699757366
334,334,nouadhibou,20.9310,-17.0347,22.08,58,33,4.14,MR,1699757408
416,416,santa ana,33.7456,-117.8678,22.29,43,0,2.06,US,1699757047


In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df
hotel_df = ideal_city_data_df_clean[["City","Country","Lat","Lng","Humidity"]].copy()

# Adding an empty column "Hotel Name"
hotel_df["Hotel Name"] =""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
11,pathalgaon,IN,22.5667,83.4667,50,
137,khairagarh,IN,26.9500,77.8167,55,
142,kidal,ML,18.4411,1.4078,11,
160,zouerate,MR,22.7187,-12.4521,24,
173,singtam,IN,27.2333,88.5000,45,
218,bairnsdale,AU,-37.8333,147.6167,51,
262,agadez,NE,19.7500,10.2500,18,
271,dourbali,TD,11.8091,15.8632,54,
334,nouadhibou,MR,20.9310,-17.0347,58,
416,santa ana,US,33.7456,-117.8678,43,


In [23]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pathalgaon - nearest hotel: No hotel found
khairagarh - nearest hotel: No hotel found
kidal - nearest hotel: No hotel found
zouerate - nearest hotel: فندق تيرس زمور
singtam - nearest hotel: Hotel Anand
bairnsdale - nearest hotel: Main hotel
agadez - nearest hotel: No hotel found
dourbali - nearest hotel: No hotel found
nouadhibou - nearest hotel: Hotel valencia
santa ana - nearest hotel: Holiday Inn Express
wamba - nearest hotel: No hotel found
kita - nearest hotel: Hotel Chats Rouges


,City,Country,Lat,Lng,Humidity,Hotel Name
11,pathalgaon,IN,22.5667,83.4667,50,No hotel found
137,khairagarh,IN,26.9500,77.8167,55,No hotel found
142,kidal,ML,18.4411,1.4078,11,No hotel found
160,zouerate,MR,22.7187,-12.4521,24,فندق تيرس زمور
173,singtam,IN,27.2333,88.5000,45,Hotel Anand
218,bairnsdale,AU,-37.8333,147.6167,51,Main hotel
262,agadez,NE,19.7500,10.2500,18,No hotel found
271,dourbali,TD,11.8091,15.8632,54,No hotel found
334,nouadhibou,MR,20.9310,-17.0347,58,Hotel valencia
416,santa ana,US,33.7456,-117.8678,43,Holiday Inn Express


In [24]:
# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points("Lng", "Lat",
                                      geo=True,
                                      tiles="OSM",  
                                      color= "City",
                                      size = "Humidity",
                                      hover_cols = ["Hotel Name", "Country"],
                                      frame_width= 900,
                                      frame_height = 600)
# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)